In [ ]:
!git clone https://github.com/ko-nlp/Kopara
!python setup.py install
!pip install Korpora

from Korpora import Korpora
!wget https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv

Cloning into 'Kopara'...
fatal: could not read Username for 'https://github.com': No such device or address
python3: can't open file '/content/setup.py': [Errno 2] No such file or directory
--2024-12-01 11:51:01--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889842 (869K) [text/plain]
Saving to: ‘ChatbotData.csv.1’

ChatbotData.csv.1   100%[===================>] 868.99K  --.-KB/s    in 0.007s  

2024-12-01 11:51:01 (123 MB/s) - ‘ChatbotData.csv.1’ saved [889842/889842]



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = pd.read_csv('ChatbotData.csv')

print(data.head())
print('데이터 개수:', len(data))

data = data.dropna()

questions = data['Q'].tolist()
answers = data['A'].tolist()
labels = data['label'].tolist() #일상:0, 사랑:1, 이별:2

                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0
데이터 개수: 11823


In [ ]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r"[^0-9가-힣a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

questions = [preprocess_sentence(q) for q in questions]
answers = [preprocess_sentence(a) for a in answers]


In [ ]:
START_TOKEN = '<sos> '
END_TOKEN = ' <eos>'

answers = [START_TOKEN + a + END_TOKEN for a in answers]


In [ ]:
tokenizer = Tokenizer(filters='', oov_token='<UNK>')
tokenizer.fit_on_texts(questions + answers)

VOCAB_SIZE = len(tokenizer.word_index) + 1
print('단어 사전 크기:', VOCAB_SIZE)


단어 사전 크기: 21746


In [ ]:
MAX_LENGTH = 40  # 최대 문장 길이 설정

# 시퀀스 변환
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

# 패딩
question_inputs = pad_sequences(question_sequences, maxlen=MAX_LENGTH, padding='post')
answer_inputs = pad_sequences(answer_sequences, maxlen=MAX_LENGTH, padding='post')


In [ ]:
answer_targets = [seq[1:] for seq in answer_sequences]
answer_targets = pad_sequences(answer_targets, maxlen=MAX_LENGTH, padding='post')


In [ ]:
from sklearn.model_selection import train_test_split

train_q, val_q, train_a_in, val_a_in, train_a_out, val_a_out = train_test_split(
    question_inputs, answer_inputs, answer_targets, test_size=0.1)


In [ ]:
embedding_dim = 256
units = 512


In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Input

# 인코더 입력
encoder_inputs = Input(shape=(None,), name='encoder_inputs')

# 임베딩 레이어
enc_emb = Embedding(VOCAB_SIZE, embedding_dim)(encoder_inputs)

# LSTM 레이어
encoder_lstm = LSTM(units, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# 인코더 상태 저장
encoder_states = [state_h, state_c]


In [ ]:
# 디코더 입력
decoder_inputs = Input(shape=(None,), name='decoder_inputs')

# 임베딩 레이어
dec_emb_layer = Embedding(VOCAB_SIZE, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

# LSTM 레이어
decoder_lstm = LSTM(units, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# 출력 레이어
from tensorflow.keras.layers import Dense

decoder_dense = Dense(VOCAB_SIZE, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# 감정 분석 레이어
emotion_dense = Dense(3, activation='softmax', name='emotion_dense')
emotion_output = emotion_dense(encoder_outputs)  # 인코더의 출력을 입력으로 사용

In [ ]:
from tensorflow.keras.models import Model

model = Model([encoder_inputs, decoder_inputs], [decoder_outputs, emotion_output])
model.compile(optimizer='adam',
              loss={'decoder_dense': 'sparse_categorical_crossentropy', 'emotion_dense': 'sparse_categorical_crossentropy'},
              loss_weights={'decoder_dense': 1.0, 'emotion_dense': 0.5})
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, None, 256)      │      5,566,976 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, None, 256)      │      5,566,976 │ decoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_lstm (LSTM)       │ [(None, 512), (None,   │      1,574,912 │ embedding[0][0]        │
│                           │ 512), (None, 512)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_lstm (LSTM)       │ [(None, None, 512),    │      1,574,912 │ embedding_1[0][0],     │
│                           │ (None, 512), (None,    │                │ encoder_lstm[0][1],    │
│                           │ 512)]                  │                │ encoder_lstm[0][2]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_dense (Dense)     │ (None, None, 21746)    │     11,155,698 │ decoder_lstm[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ emotion_dense (Dense)     │ (None, 3)              │          1,539 │ encoder_lstm[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 25,441,013 (97.05 MB)

 Trainable params: 25,441,013 (97.05 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#질문, 답변 입력 및 타겟 시퀀스
train_q, val_q, train_a_in, val_a_in, train_a_out, val_a_out, train_labels, val_labels = train_test_split(
    question_inputs, answer_inputs, answer_targets, labels, test_size=0.1)

# 타겟 데이터에 차원 추가
train_a_out = np.expand_dims(train_a_out, -1)
val_a_out = np.expand_dims(val_a_out, -1)

# 모델 학습
history = model.fit(
    {'encoder_inputs': np.array(train_q), 'decoder_inputs': np.array(train_a_in)},  # Pass inputs as a dictionary
    {'decoder_dense': train_a_out, 'emotion_dense': np.array(train_labels)},  # Convert labels to NumPy array
    validation_data=({'encoder_inputs': np.array(val_q), 'decoder_inputs': np.array(val_a_in)},
                     {'decoder_dense': val_a_out, 'emotion_dense': np.array(val_labels)}),
    batch_size=64,
    epochs=10)

Epoch 1/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 34s 151ms/step - decoder_dense_loss: 2.4395 - emotion_dense_loss: 0.5440 - loss: 2.9835 - val_decoder_dense_loss: 0.8999 - val_emotion_dense_loss: 0.5459 - val_loss: 1.4496
Epoch 2/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 33s 133ms/step - decoder_dense_loss: 0.8592 - emotion_dense_loss: 0.5365 - loss: 1.3957 - val_decoder_dense_loss: 0.8880 - val_emotion_dense_loss: 0.5366 - val_loss: 1.4282
Epoch 3/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 42s 139ms/step - decoder_dense_loss: 0.8223 - emotion_dense_loss: 0.5389 - loss: 1.3612 - val_decoder_dense_loss: 0.8897 - val_emotion_dense_loss: 0.5372 - val_loss: 1.4302
Epoch 4/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 42s 143ms/step - decoder_dense_loss: 0.7849 - emotion_dense_loss: 0.5379 - loss: 1.3229 - val_decoder_dense_loss: 0.8995 - val_emotion_dense_loss: 0.5376 - val_loss: 1.4403
Epoch 5/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 24s 144ms/step - decoder_dense_loss: 0.7530 - emotion_dense_loss: 0.5349 - loss: 1.2879 - val_decoder_dense_

In [ ]:
# 인퍼런스 인코더
encoder_model = Model(encoder_inputs, [encoder_states, emotion_output])

# 인퍼런스 디코더 입력
decoder_state_input_h = Input(shape=(units,))
decoder_state_input_c = Input(shape=(units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs)

decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [66]:
def decode_sequence(input_seq):
    # 인코더를 통해 상태 벡터와 감정 예측 얻기
    encoder_outputs = encoder_model.predict(input_seq)
    states_value, predicted_emotion = encoder_outputs[0], encoder_outputs[1]

    # 감정 예측 결과를 기반으로 맞춤형 응답 제공
    emotion_index = np.argmax(predicted_emotion[0])
    #emotion_text = {0: "일상", 1: "긍정", 2: "부정"}[emotion_index]
    emotion_text = {0: "", 1: "", 2: ""}[emotion_index]

    # 목표 시퀀스 초기화 (시작 토큰의 인덱스)
    start_token_index = tokenizer.word_index.get(START_TOKEN.strip(), 1)  # OOV 방지를 위해 기본값 1 설정
    target_seq = np.array([[start_token_index]])  # 모양을 (1, 1)로 지정

    # 결과 저장용 변수
    #decoded_sentence = f"[{emotion_text}] "
    decoded_sentence = ""

    while True:
        # 디코더 모델에 입력 상태 전달 (감정 출력 제거)
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과에서 단어 인덱스 추출
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, '<UNK>')

        # 종료 조건 확인: 종료 토큰이거나 최대 길이 초과
        if (sampled_word == END_TOKEN.strip()) or (len(decoded_sentence.split()) > MAX_LENGTH):
            break

        # 결과 문장에 단어 추가
        decoded_sentence += ' ' + sampled_word

        # 목표 시퀀스 업데이트 (모양을 (1, 1)로 유지)
        target_seq = np.array([[sampled_token_index]])

        # 상태 업데이트
        states_value = [h, c]

    return decoded_sentence.strip()


In [56]:
import torch
from transformers import BertTokenizer
import torch.nn.functional as F

# 감정 분류 모델 정의 및 로드
class BERTClassifier(torch.nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=3, dr_rate=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = torch.nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        _, pooled_output = self.bert(
            input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=False
        )
        return self.classifier(pooled_output)

from transformers import BertModel
bert = BertModel.from_pretrained('bert-base-multilingual-cased')

# 모델 초기화
bert_model = BERTClassifier(bert=bert)

# state_dict 로드
checkpoint = torch.load('emotion_classification_model.pt')

# 필요한 키만 로드
model_state_dict = bert_model.state_dict()
filtered_checkpoint = {k: v for k, v in checkpoint.items() if k in model_state_dict and v.size() == model_state_dict[k].size()}
model_state_dict.update(filtered_checkpoint)

# 업데이트된 state_dict 로드
bert_model.load_state_dict(model_state_dict)
bert_model.eval()

b_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 감정 분류 함수
def predict_emotion(text):
    inputs = b_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    with torch.no_grad():
        outputs = bert_model(**inputs)  # 올바른 모델 이름 사용
        logits = outputs
        probs = F.softmax(logits, dim=-1)
        predicted_label = torch.argmax(probs, dim=-1).item()
    labels = ['일상(_)', '사랑(긍정)', '이별(부정)']
    return labels[predicted_label]

<ipython-input-56-74d18efe7382>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('emotion_classification_model.pt')


In [67]:

def chatbot_response(input_sentence):
    # 1. 입력 문장 전처리
    input_sentence_processed = preprocess_sentence(input_sentence)
    input_sequence = tokenizer.texts_to_sequences([input_sentence_processed])
    input_padded = pad_sequences(input_sequence, maxlen=MAX_LENGTH, padding='post')

    # 2. 감정 분류
    emotion = predict_emotion(input_sentence)  # 감정 분류 모델 호출

    # 3. 응답 생성
    response = decode_sequence(input_padded)  # 기존 응답 생성 로직

    # 4. 감정 분류 결과 포함
    final_response = f"[{emotion}] {response}"  # 감정 결과를 대괄호로 표시
    #final_response = f"{response}"

    return final_response


In [ ]:
# 예시 테스트

while True:
    input_sentence = input('You: ')
    if input_sentence == 'exit':
        break
    response = chatbot_response(input_sentence)
    print('Chatbot:', response)


You: 오늘 선생님께 혼이 나서 슬퍼
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Chatbot: [일상]  다른 위한 위해 건 없어요.
You: 어제 연인이랑 헤어졌어
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Chatbot: [일상]  제가 생각을 마세요.
You: 오늘은 아주 특별한 날이라 설레
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Chatbot: [일상]  자신과 진실로 존중하고 그런 것
You: 친구랑 다퉜어
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/st

In [68]:
input_sentence = '친구랑 크게 다퉜어 그래서 어쩔건데'
user_input = '친구랑 크게 다퉜어'
response = chatbot_response(input_sentence)
print('Chatbot:', response)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Chatbot: [이별(부정)] 좋은
